In [4]:
import numpy as np
import pandas as pd
import nltk
import gensim
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()
from gensim import corpora, models
import re
stemmed_tokens=[]
import string
from nltk.corpus import stopwords

In [5]:
articles_data=pd.read_csv("news_articles.csv")
article=list(articles_data['Content'])

In [6]:
for i in range(4831):
    raw=article[i].lower()
    raw=re.sub(r'\d+', '', raw)        #removing punctutation
    word_tokens = tokenizer.tokenize(raw)   #tokens
    stpwrd_sentence = [w for w in word_tokens if not w in stop_words] #remove stopwords is,am,are..... common words
    lemmatised_words=[lemmatizer.lemmatize(w) for w in stpwrd_sentence]  #lemmitizer to get root form of word
    stemmed_words=[stemmer.stem(i) for i in lemmatised_words]            #stemming
    stemmed_tokens.append(stemmed_words)   


In [7]:
dictionary=corpora.Dictionary(stemmed_tokens)
corpus=[dictionary.doc2bow(d) for d in stemmed_tokens]             #count how many times word occur
#corpus                     #lists of document 1 to n 

In [8]:
ldamodel=gensim.models.ldamodel.LdaModel(corpus,num_topics=4,id2word=dictionary,passes=20,minimum_probability=0.0)

In [9]:
topics=[]
for i in range(4831):
    item=ldamodel[corpus[i]]                      #gives probabiloty of i document in topics 0,1,2,3
    item=np.array(item).T                       #for computation convert list to array and transpose to get 2*4
    item=item[1]                                  #get probabilites  1*4
    topics.append(item)                               # append  probabilites 
topics_array=np.array(topics)

In [10]:
#kmeans 
import sklearn
from sklearn.cluster import KMeans
kmeans=sklearn.cluster.KMeans(4).fit(topics)    #clusters using topics(probabilty of matric  n*4  (n=documents,4=topics))
clusters=kmeans.labels_

In [11]:
#add length column
length=[]
for i in range(4831):
    length.append(len(articles_data["Content"][i]))
    
articles_data["Length"]=length
#add average time
ti=[]
for i in range(4831):
    ti.append(int(length[i]/5))
articles_data["avg_Time"]=ti

In [154]:
noofusers=int(input("enter users: "))

enter users: 20


# 

In [155]:
#add user ids
pls=[]
for i in range(1,noofusers+1):
    for j in range(8):
        p=1*i
        pls.append(p)
    

In [156]:
datathrough=pd.DataFrame()                                                   #articles alloted to users
for i in range (noofusers):
    for i in range(4):
        user=(articles_data[clusters==i][:2])
        datathrough=datathrough.append(user)

In [157]:
datathrough.to_pickle("articlethrough.pkl")
#pd.read_pickle("articlethrough.pkl") read from pickle  
#datathrough

In [158]:
#genrative

In [159]:
 def timefxn(user_profile):
    time1=np.random.normal(20,0.3,2000)
    time2=np.random.normal(30,0.3,2000)
    time3=np.random.normal(100,0.3,831)
    total_time=np.append(time1,time2)
    time=np.append(total_time,time3)
    actual=np.random.choice(time,len(user_profile["Article_Id"]))
    return actual

In [160]:
def checkfxn(user_profile,hey,actual,noofusers):
    check=[]            #yes or no and add time
    actual_time=[]
    for j in range(1,noofusers+1):
        for i in range(8):
            if list(user_profile["Article_Id"][user_profile["User_id"]==j])[i] in list(hey['Article_Id'][hey["User_id"]==j]):
                check.append("YES")
                actual_time.append(int(actual[i*j]))
            
            else:
                check.append("NO")
                actual_time.append(0)
    return check,actual_time

In [161]:
def addweight(user_info):
    import math             #finding weight
    user_info["ratio"]=user_info["actual_time"]/user_info["average_Time"]
    weight=[]
    for i in user_info["ratio"]:
        if i==0:
            weight.append(0)
        else:
            weight.append(1/(1+math.exp(-i)))
    user_info["weight"]=weight
    user_info.drop(["ratio"],axis=1,inplace=True)
    probability=[]
    for i in range(len(user_info["articleID"])):
        yo=weight[i]*topics[user_info["articleID"][i]]
        probability.append(yo)
    user_info["weight*prob"]=probability
    
    return user_info

In [162]:
def ourfxn(noofusers,datathrough):                                          #fxn on through artciles
    hey=pd.DataFrame()
    datathrough["User_id"]=pls
    user_prof=datathrough
    user_profile=user_prof[["Article_Id",'Title','Length',"User_id","avg_Time"]].copy()
    number=[]
    for i in range(noofusers):
        no_of_articles=np.random.poisson(3,1)[0]
        number.append(no_of_articles)
    time=timefxn(user_profile)
    

    for i in range(1,noofusers+1):
        hey=hey.append(user_profile[user_profile["User_id"]==i].sample(number[i-1])) #throw n samples from userr(dataframe of 8 articles ,2 from each topics)
    #print(user_profile)
    check,actual_time=checkfxn(user_profile,hey,time,noofusers)
    user_info=pd.DataFrame({"articleID":list(user_profile["Article_Id"]),"Title":list(user_profile["Title"]),"Length":list(user_profile["Length"]),"User_id":list(user_profile["User_id"]),"check":check,"average_Time":list(user_profile["avg_Time"]),"actual_time":actual_time})
    user_info=addweight(user_info)
    return user_info

In [163]:
#new_info=ourfxn(2)
#new_info.to_pickle("userinfo.pkl")

In [164]:
from sklearn.metrics.pairwise import cosine_similarity         #articles suggested after cosimlarity
def cosim(noofusers,user_info):
    b=[]
    #user_info=pd.read_pickle("userinfo.pkl")
    userspro=[]                                 #profile of users after summing the prob
    for i in range(1,noofusers+1):
        userspro.append(np.sum(user_info["weight*prob"][user_info["User_id"]==i]))
    suggestarticles=[]
    for i in userspro:
        suggestarticles.append(np.argsort(cosine_similarity(topics,i.reshape(1,-1)).flatten())[-8:][::-1])
    for i in range(noofusers):
        for j in range(8):
            b.append(suggestarticles[i][j])
            
    return articles_data.iloc[b]  

In [165]:
noofsession=int(input("sessions: "))


sessions: 


ValueError: invalid literal for int() with base 10: ''

In [166]:
for i in range(1,3):
    if i ==1:
        data=ourfxn(noofusers,datathrough)
        data["session"]=np.ones(8*noofusers)*i
        data.to_pickle("newdata.pkl")
    if i ==2 :
        data1=pd.read_pickle("newdata.pkl")
        data2=cosim(noofusers,data1)
        data3=ourfxn(noofusers,data2)
        data3["session"]=np.ones(8*noofusers)*i
        data=data1.append(data3)
        data.to_pickle("newdata.pkl")
        
        
        
        
        
    
        

     Article_Id                                              Title  Length  \
1             1  Pratibha Tiwari molested on busy road   Saath ...    1215   
27           27   Star Trek  actor Anton Yelchin dead in fatal ...     814   
6             6  Infosys shares likely to fall on Tuesday after...    2104   
630         630  Baahubali  Bajarangi Bhaijaan writer pens stor...      42   
0             0  14 dead after bus falls into canal in Telangan...    1355   
2             2  US  South Korea begin joint military drill ami...    1369   
870         870  Gionee S6s with 8MP front camera  LED flash la...    1150   
871         871  Batman-inspired Samsung Galaxy Note7 edition i...     813   
1             1  Pratibha Tiwari molested on busy road   Saath ...    1215   
27           27   Star Trek  actor Anton Yelchin dead in fatal ...     814   
6             6  Infosys shares likely to fall on Tuesday after...    2104   
630         630  Baahubali  Bajarangi Bhaijaan writer pens stor.

G:\Newfolder\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


      Article_Id                                              Title  Length  \
1167        1167  Top 5 torrent sites for 2016  KickassTorrents ...    2094   
1868        1868  Tip to Put Your Baby to Sleep in Seconds  Nath...    1151   
862          862  Scappi for Chennai  Here s how to help Chennai...     878   
1017        1017  IsoHunt resurrects KickassTorrents  KAT  via c...    1948   
1180        1180  Twitter expands verification application proce...    1802   
1023        1023  After The Pirate Bay  KickassTorrents  KAT  go...    2842   
2172        2172  Navroz Mubarak Iranian New Year  Google Celebr...    1085   
1165        1165  Swedish court orders shutdown of The Pirate Ba...    1602   
1101        1101  Make in India  Ringing Bells set to launch che...    1378   
2662        2662  Google revamps Developer Program Policy  bring...    1799   
1391        1391  Project SkyBender  Google working on solar-pow...    2713   
1275        1275  Black Friday 2015 deals  OnePlus j

In [167]:
pd.read_pickle("newdata.pkl")

,articleID,Title,Length,User_id,check,average_Time,actual_time,weight,weight*prob,session
0,1,Pratibha Tiwari molested on busy road Saath ...,1215,1,NO,243,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
1,27,Star Trek actor Anton Yelchin dead in fatal ...,814,1,YES,162,30,0.546164,"[0.014963038986812612, 0.38224224792993655, 0....",1.0
2,6,Infosys shares likely to fall on Tuesday after...,2104,1,NO,420,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
3,630,Baahubali Bajarangi Bhaijaan writer pens stor...,42,1,NO,8,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
4,0,14 dead after bus falls into canal in Telangan...,1355,1,NO,271,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
5,2,US South Korea begin joint military drill ami...,1369,1,YES,273,29,0.526532,"[0.0009989009519817316, 0.028544783167520998, ...",1.0
6,870,Gionee S6s with 8MP front camera LED flash la...,1150,1,NO,230,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
7,871,Batman-inspired Samsung Galaxy Note7 edition i...,813,1,YES,162,30,0.546164,"[0.48150148250433594, 0.039212963566372064, 0....",1.0
8,1,Pratibha Tiwari molested on busy road Saath ...,1215,2,NO,243,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
9,27,Star Trek actor Anton Yelchin dead in fatal ...,814,2,NO,162,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
